In [3]:
#! /usr/bin/python3
import praw
import pandas as pd
import datetime as dt

<h1>Part 1: Reddit Data Collection </h1>
I will first be extracting 12 flairs. For each flair, I want to extract 100 reddit posts. For each post, I will be extracting the following data: Title, Body, Comments, URL, Score, Comments Number, Create date.
One important choice I had to make was how many comments to extract. I have decided to perform a Breadth First Search, taking only long comments (>200 chars). The rationale is explained during Exploratory Data Analysis. Some posts naturally have far more comments (thousands) than others (less than 10). Thus, I have not extracted all comments. Instead I have employed a heuristic that doesn't replace the 'View more comments' option beyond a certain number of times for posts with more comments.

In [17]:
flairs = ["AskIndia", "Non-Political", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "Coronavirus", "CAA-NRC-NPR"]
reddit = praw.Reddit(client_id='EGV0l4iNPHX3zw', \
                     client_secret='cfZJqAtGpacEHzjzo6ala9IXhOc', \
                     user_agent='flairdetectorscrape', \
                     username='logisbase2', \
                     password='W3codeforlife')
subreddit = reddit.subreddit('india')
cols = {"flair":[], "title":[], "body":[], "comments":[], "url":[], "score":[], "comms_num": [], "created": []}
flair_num = 0
for flair in flairs:
    flair_num+=1
    print("Flair Number: " + str(flair_num) + " " + flair)
    
    flair_comms=0
    get_posts = subreddit.search("flair:"+flair, limit=100)

    post_num = 0
    for post in get_posts:
        post_num+=1
        if(post_num%10==0):
            print(post_num) 
        cols["flair"].append(flair)
        cols["title"].append(post.title)
        cols["body"].append(post.selftext)
        cols["url"].append(post.url)
        cols["score"].append(post.score)
        cols["comms_num"].append(post.num_comments)
        cols["created"].append(post.created)
        
        comm_more = None
        if(post.num_comments > 2000):
            comm_more = 3
        elif(post.num_comments > 500):
            comm_more = 7
        elif(post.num_comments > 100):
            comm_more = 15
        char_threshold = 200
        count_threshold = 20
        count = 0
        comment_list = []
        post.comments.replace_more(limit=comm_more)
        comment_queue = post.comments[:]  # Start with top-level comments
        while comment_queue:
            comment = comment_queue.pop(0)
            if(len(comment.body) > char_threshold):
                count+=1
                flair_comms+=1
            comment_list.append(comment.body)
            if(count >= count_threshold):
                break
        cols["comments"].append(comment_list)
    print(flair_comms)

Flair Number: 1 AskIndia
10
20
30
40
50
60
70
80
90
100
663
Flair Number: 2 Non-Political
10
20
30
40
50
60
70
80
90
100
527
Flair Number: 3 Scheduled
10
20
30
40
50
60
70
80
90
100
1660
Flair Number: 4 Photography
10
20
30
40
50
60
70
80
90
100
63
Flair Number: 5 Science/Technology
10
20
30
40
50
60
70
80
90
100
178
Flair Number: 6 Politics
10
20
30
40
50
60
70
80
90
100
741
Flair Number: 7 Business/Finance
10
20
30
40
50
60
70
80
90
100
172
Flair Number: 8 Policy/Economy
10
20
30
40
50
60
70
80
90
100
336
Flair Number: 9 Sports
10
20
30
40
50
60
70
80
90
100
84
Flair Number: 10 Food
10
20
30
40
50
60
70
80
90
100
158
Flair Number: 11 Coronavirus
10
20
30
40
50
60
70
80
90
100
81
Flair Number: 12 CAA-NRC-NPR
10
20
30
40
60


In [21]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

topics_data = pd.DataFrame(cols)
_timestamp = topics_data["created"].apply(get_date)
topics_data = topics_data.assign(timestamp = _timestamp)
del topics_data['created']
topics_data.to_csv('reddit-india-data.csv', index=False)

The previous cell converts the time from UNIX to UTC and loads the data in a csv. <br>
Unfortunately CAA-NRC-CAB has only 40-50 posts and thus will not be used as a flair in further analysis.